In [1]:
import numpy as np
from scipy.stats import uniform, norm, binom_test

In [2]:


def calc_bootstrap_confidence_interval(
    sample, statistic, method="quantile",
    n_resamples=1000, significance_level=0.05
):
    assert method in {"quantile", "norm"};
    taken_indices = np.random.randint(0, len(sample), size=(n_resamples, len(sample)))
    bootstrapped_samples = sample[taken_indices]
    bootstrapped_stats = statistic(bootstrapped_samples, axis=1)

    stat_val = statistic(sample)

    if method == "quantile":
        mean_val = np.mean(bootstrapped_stats)
        upper_quantile = np.percentile(bootstrapped_stats, (1 - significance_level / 2) * 100)
        lower_quantile = np.percentile(bootstrapped_stats, (significance_level / 2) * 100)
        lower_bound = stat_val - (upper_quantile - mean_val)
        upper_bound = stat_val + (mean_val - lower_quantile)
    else:
        dev = np.std(bootstrapped_stats, ddof=1) * norm.ppf(1 - significance_level / 2)
        lower_bound = stat_val - dev
        upper_bound = stat_val + dev

    return (lower_bound, upper_bound)


In [3]:
def unbiased_variance(sample, axis=None):
    return np.var(sample, axis=axis, ddof=1)

def is_in(pt, interval):
    return pt > interval[0] and pt < interval[1]

n_simulations = 10000
sample_size = 100
distr = uniform(0, 1)
mistakes_count = 0
significance_level = 0.05
method = "norm"

for i in range(n_simulations):
    sample = distr.rvs(sample_size)
    interval = calc_bootstrap_confidence_interval(
        sample, unbiased_variance, method=method,
        significance_level=significance_level)
    if not is_in(distr.var(), interval):
        mistakes_count += 1

print(mistakes_count / n_simulations)
binom_test(mistakes_count, n_simulations, p=significance_level)


0.0525


0.251296217709316

### Вывод

Бутстрэп работает не идеально, но количество ошибок достаточно близко к установленному уровню значимости (хоть и получается несколько больше в данном случае). Бутстрэп работает тем хуже чем меньше разме